In [1]:
import pandas as pd
import json
import numpy as np
import hashlib
import re
from sentence_transformers import SentenceTransformer
import faiss


/opt/anaconda3/envs/safevision/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('./RuBQ_2.0_paragraphs.json', 'r') as f:
    data = json.load(f)

In [3]:
df = pd.DataFrame(data)
print(df.shape)
df.head()

(56952, 3)


,uid,ru_wiki_pageid,text
0,0,58311,ЦСКА — советский и российский профессиональный...
1,1,58311,В первом сезоне в составе Континентальной хокк...
2,2,58311,В межсезонье 1992 года «армейскую» команду пок...
3,3,58311,"Однако ни Тихонов, ни Гущин, не согласились с ..."
4,4,58311,ЦСКА Александра Волчкова сезон 1996/97 провел ...


In [4]:
def data_info(df):
    print(f"Всего текстов: {len(df)}")
    print(f"Средняя длина текста: {df['text'].str.len().mean():.0f} символов")
    print(f"Мин. длина текста: {df['text'].str.len().min():.0f} символов")
    print(f"Макс. длина текста: {df['text'].str.len().max():.0f} символов")

data_info(df)

Всего текстов: 56952
Средняя длина текста: 449 символов
Мин. длина текста: 1 символов
Макс. длина текста: 11010 символов


In [5]:
import logging
logging.basicConfig(filename='data_quality.log', level=logging.INFO)

logger = logging.getLogger(__name__)

def compute_text_hash(text):
    return hashlib.sha256(text.strip().lower().encode('utf-8')).hexdigest()


def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    return text

def check_data_quality(df, min_len=10):
    # Удаление лишних пробелов
    df['text'] = df['text'].str.strip()

    # Проверка на пустые документы
    empty_docs = df[(df['text'].str.strip() == '')]
    logger.info(f'Пустых документов: {len(empty_docs)}')

    # Проверка дубликатов
    duplicate_uids = df[df.duplicated(subset=['uid'], keep=False)]
    logger.info(f"Дубликатов по uid: {len(duplicate_uids)}")

    df['text_hash'] = df['text'].apply(compute_text_hash)
    duplicate_texts = df[df.duplicated(subset=['text_hash'], keep=False)]
    logger.info(f"Дубликатов текстов: {len(duplicate_texts)}")

    # Проверка на минимальную длину
    short_texts = df[df['text'].str.len() < min_len]
    logger.info(f'Текстов длиной меньше {min_len}: {len(short_texts)}')

    res = {}
    res['empty_docs'] = {'count': len(empty_docs), 'data': empty_docs.to_dict(orient='records')}
    res['duplicate_uids'] = {'count': len(duplicate_uids), 'data': duplicate_uids.to_dict(orient='records')}
    res['duplicate_texts'] = {'count': len(duplicate_texts), 'data': duplicate_texts.to_dict(orient='records')}
    res['short_texts'] = {'count': len(short_texts), 'data': short_texts.to_dict(orient='records')}

    # Очистка данных
    short_texts_uids = short_texts['uid']
    df_clean = df[~df['uid'].isin(short_texts_uids)]

    df_clean = df_clean.drop_duplicates(subset=['text_hash'], keep='first')
    df_clean = df_clean.drop('text_hash', axis=1)

    # Нормализация
    df_clean['text'] = df_clean['text'].apply(normalize_text)
    
    return res, df_clean


data_quality, df_clean = check_data_quality(df)

with open('data_quality_log.json', 'w') as f:
    json.dump(data_quality, f, ensure_ascii=False)

df_clean.to_json('good_texts.json', orient='records', force_ascii=False, indent=4)

In [6]:
def create_embeddings(texts, model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", batch_size=32):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(
        texts,
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_numpy=True
    )

    return embeddings

def create_faiss(embeddings, index_path="data/RuBQ_index.index"):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)

    index.add(np.array(embeddings, dtype=np.float32))

    faiss.write_index(index, index_path)

In [ ]:
embeddings = create_embeddings(df_clean['text'].tolist())
create_faiss(embeddings)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Batches: 100%|██████████| 1773/1773 [01:35<00:00, 18.56it/s]


In [ ]:
np.save("data/my_embeddings.npy", embeddings)

In [2]:
# Проверка на корректность поиска
index = faiss.read_index("data/RuBQ_index.index")
embeddings = np.load("data/my_embeddings.npy")

query = embeddings[42].reshape(1, -1)  

D, I = index.search(query, k=5)

print("Индексы ближайших соседей:", I)
print("Расстояния:", D)

print("Первый найденный индекс:", I[0][0])


Индексы ближайших соседей: [[   42 11926    41 11925 46356]]
Расстояния: [[0.42342547 4.188086   4.77685    4.912714   4.944633  ]]
Первый найденный индекс: 42


In [3]:
with open('data/good_texts.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)
data = df['text'].tolist()

In [4]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

query = "был сыгран первый в мировой истории футбольный матч между клубными командами"

query_embedding = model.encode([query], convert_to_numpy=True).astype("float32")

k = 5
distances, indices = index.search(query_embedding, k)

for i, idx in enumerate(indices[0]):
    print(f"{i+1}. {data[idx]}\n   (расстояние: {distances[0][i]:.4f})")


1. 26 декабря 1860 года между «Шеффилдом» и «Хэлламом» был сыгран первый в мировой истории футбольный матч между клубными командами.
   (расстояние: 5.6396)
2. Впервые за 88 лет хозяйкой мирового футбольного первенства окажется сборная, ранее никогда не участвовавшая в данном соревновании. Ранее такое случалось только в 1934 году, когда ЧМ проходил в Италии, кстати, взявшей тогда Кубок мира (таким образом, если Катар выиграет ЧМ-2022, то ещё и впервые за почти девять десятилетий футбольная сборная впервые принимающая матчи Кубка мира и одновременно вообще впервые участвующая в этом турнире, станет его победителем). Также в родных стенах дебютировала на чемпионатах мира и сборная Уругвая — однако здесь речь идет о самом первом мундиале в истории, проходившем в 1930 году.
   (расстояние: 5.6841)
3. В этой группе было четыре команды: Аргентина, Чили, Франция и Мексика. Один из двух первых матчей чемпионата мира — между Францией и Мексикой — начался 13 июля в 15:00 на стадионе «Поситос», н